In [16]:
# 9.20 利用函数注解实现方法重载
"""问题
你已经学过怎样使用函数参数注解，那么你可能会想利用它来实现基于类型的方法重载。 但是你不确定应该怎样去实现（或者到底行得通不）。

解决方案
本小节的技术是基于一个简单的技术，那就是Python允许参数注解，代码可以像下面这样写："""
class Spam:
    def bar(self, x:int, y:int):
        print('Bar 1', x, y)
        
    def bar(self, s:str, n:int=0):
        print('Bar 2', s, n)

s = Spam()
s.bar(2, 3)
s.bar('Hello')
s.bar(2, 'hello')

Bar 2 2 3
Bar 2 Hello 0
Bar 2 2 hello


In [7]:
"""下面是我们第一步的尝试，使用到了一个元类和描述器："""
import inspect
import types

class MultiMethod:
    """
    Represents a single multimethod.
    """
    def __init__(self, name):
        self._methods = {}
        self.__name__ = name
        
    def register(self, meth):
        '''
        Register a new method as a multimethod
        '''
        sig = inspect.signature(meth)
        types = []
        for name, parm in sig.parameters.items():
            if name == 'self':
                continue
            if parm.annotation is inspect.Parameter.empty:
                raise TypeError("Argument {} must be annotated with a type.".format(name))
            if not isinstance(parm.annotation, type):
                raise TypeError("Argument {} annotation must be a type.".format(name))
            if parm.default is not inspect.Parameter.empty:
                self._methods[tuple(types)] = meth
            types.append(parm.annotation)
        self._methods[tuple(types)] = meth
        
    def __call__(self, *args):
        '''
        Call a method based on type signature of the arguments
        '''
        types = tuple(type(arg) for arg in args[1:])
        meth = self._methods.get(types, None)
        if meth:
            return meth(*args)
        else:
            raise TypeError("No matching method for types {}".format(types))
            
    def __get__(self, instance, cls):
        '''
        Descriptor method needed to make calls work in a class
        '''
        if instance is not None:
            return types.MethodType(self, instance)
        else:
            return self
        
class MultiDict(dict):
    '''
    Special dictionary to build multimethods in a metaclass
    '''
    def __setitem__(self, key, value):
        if key in self:
            current_value = self[key]
            if isinstance(current_value, MultiMethod):
                current_value.register(value)
            else:
                mvalue = MultiMethod(key)
                mvalue.register(current_value)
                mvalue.register(value)
                super().__setitem__(key, mvalue)
        else:
            super().__setitem__(key, value)
            
class MultipleMeta(type):
    '''
    Metaclass that allows multiple dispatch of methods
    '''
    def __new__(cls, clsname, bases, clsdict):
        return type.__new__(cls, clsname, bases, dict(clsdict))
    
    @classmethod
    def __prepare__(cls, clsname, bases):
        return MultiDict()
    
"""为了使用这个类，你可以像下面这样写："""
class Spam(metaclass=MultipleMeta):
    def bar(self, x:int, y:int):
        print('Bar 1:', x, y)

    def bar(self, s:str, n:int = 0):
        print('Bar 2:', s, n)
    
# Example: overloaded __init__
import time

class Date(metaclass=MultipleMeta):
    def __init__(self, year: int, month:int, day:int):
        self.year = year
        self.month = month
        self.day = day

    def __init__(self):
        t = time.localtime()
        self.__init__(t.tm_year, t.tm_mon, t.tm_mday)

In [8]:
"""下面是一个交互示例来验证它能正确的工作："""
s = Spam()

In [9]:
s.bar(2, 3)

Bar 1: 2 3


In [10]:
s.bar('hello')

Bar 2: hello 0


In [11]:
s.bar('hello', 5)

Bar 2: hello 5


In [12]:
s.bar(2, 'hello')

TypeError: No matching method for types (<class 'int'>, <class 'str'>)

In [13]:
d = Date(2012, 12, 21)

In [14]:
e = Date()

In [15]:
e.year

2019